# Plotting Strataform results

This notebook creates a cross-section along **Strataform** output. It will be handy to look at the stratigraphic stacking as well as the sediment heterogeneity in underlying layers.

First, we need to import some library. As you can see below we import the **StrataPlot** module which is a python script that you can view in the `GeolProc` directory.

In [ ]:
import numpy as np
import StrataPlot as stratVis
%matplotlib inline

# Running Strataform

To run **strataform** we need to move to the directory containing the input file. 

In [ ]:
!ls OTR/

In this first example we will use `OTRsim1.sif` located in the `OTR` folder. As for the XBeach example we will execute the code using the `!` command in front. It tells IPython that we want to use the `bash` mode. 

The command is:

In [ ]:
!cd OTR; strataform OTRsim1.sif

# Load required files

**Strataform** writes VTK files at user-defined interval. These files are in a folder which has the same name as your input file. We will load 2 files the basement to get the bottom of the stratigraphic layers and the stratigraphic record itself.

In addition you will need to add the start and time of the run based on the input file chosen parameters.

In [ ]:
!ls OTR/OTRsim1/OTRsim1

In [ ]:
filebase ='./OTR/OTRsim1/OTRsim1/OTRsim1_99_bas.vtu'
filestrati ='./OTR/OTRsim1/OTRsim1/OTRsim1_99.vtu'

# Simulation start and end times (in years)
simStart = -1000000.
simEnd = 1000000.

# Number of stratigraphic layers recorded
# This is the number at the end of the filestrati name + 1
lays = 100

# Ellapsed time between 2 stratigraphic layers (in years)
dt = (simEnd-simStart)/lays

# Read and create relevant numpy arrays for plotting
minBase,x,y,z,mgz,age = stratVis.read_VTK(filebase,filestrati,simStart)

# Visualise sea-level fluctuations

In case you have loaded a sea-level file you can visualise it from the cell below.

In [ ]:
#seafile = './strataGeo/data/sine_2cycle_rsl.sl'
#SLtime,sealevel = stratVis.read_seaLevel(seafile)

# Interpolate the unstructured dataset to a regular grid

We now interpolate the data over a regular points. We need to take care of the sedimentary hiatus from the stratigraphic records. To do the regridding we use the function `mapData_Reg`.

In [ ]:
# Set the regular grid resolution (in meters)
res = 5.0

nlays,xi,yi,zi,mzi,nxi,nyi = stratVis.mapData_Reg(lays,x,y,z,mgz,age,res,simStart,dt)

# Y-slicing the stratigraphic volume

We now extract a slice along the Y-axis using in this case the middle of the simulation which is 20 km x 20 km.

In [ ]:
# X-axis value for the Y cross-section (in meters)
posX = 500.
# sea-level final sea-level position
slvl = 0.
# Get Y cross-section
Ysec,base,sl,xID = stratVis.crossYsection(posX,res,xi,yi,nxi,nyi,minBase,slvl)

Using the cross-section defined above we plot the time layer evolution of the **Strataform** model.

In [ ]:
# Size of the figure
figS = [15,6]
# Y and Z axis clipping values
ylim = [-500,7500]
zlim = [-30,20]
# Frequency of layer outputs
layplot = 10
stratVis.plotYtime(figS,Ysec,zi,xID,base,minBase,sl,nlays,layplot,ylim,zlim)

# Visualising sediments heterogeneities along Y-axis

Using the same approach, we plot the mean grain size (in mm) for each of the stratigraphic layers. 

In [ ]:
'''
Define the number of points for mean grain size interpolation along the z axis
 - first value is Z bottom, 
 - second value is the maximum elevation of your interpolation grid
 - third parameter is the number of point along the Z axis 
 which in the case below means that we will put a point every 1 m
'''
mgzYGridRes = [-40, 20, 241]
mgzY,zY = stratVis.getYmgz(Ysec,zi,yi,mzi,nlays,xID,mgzYGridRes)

# Plot the mean grain zise along the Y-axis
figS = [15,6]
# Y and Z axis clipping values
ylim = [-500,7500]
zlim = [-30,20]
# Frequency of layer outputs
layplot = 10
stratVis.plotYmgzReef(figS,Ysec,zi,zY,mgzY,nlays,xID,ylim,zlim,minBase,sl,layplot)